In [1]:
import contextlib
import itertools
import sqlite3
from pathlib import Path

import exiftool
from tqdm.auto import tqdm

d:\workspace\e4e\fishsense\2025-03-Reef-Data-Processing\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_root = Path(r"\\e4e-nas.ucsd.edu\fishsense_data\REEF\data")
data_db = Path(r"\\e4e-nas.ucsd.edu\fishsense_data\REEF\processing.db")
exiftool_path = Path(r"C:\Users\nthui\Downloads\exiftool-13.22_64\exiftool(-k).exe")

In [3]:
with contextlib.closing(sqlite3.connect(data_db)) as con, contextlib.closing(con.cursor()) as cur:
    images = [data_root.joinpath(row[0]) for row in cur.execute('SELECT path FROM images WHERE images.camera_sn ISNULL;').fetchall()]

    with exiftool.ExifToolHelper(executable=exiftool_path.as_posix()) as et:
        for image_batch in itertools.batched(tqdm(images, 'Image Camera SN'), n=1):
            try:
                metadata = et.get_tags(image_batch, ['MakerNotes:SerialNumber'])
            except Exception as exc:
                print(exc)
                continue

            for data in metadata:
                try:
                    params = {
                                    'camera_sn': data['MakerNotes:SerialNumber'].strip(),
                                    'path': Path(data['SourceFile']).relative_to(data_root).as_posix()
                                }
                except KeyError:
                    continue
                cur.execute('UPDATE images SET camera_sn = :camera_sn WHERE images.path = :path',
                            params)
                con.commit()

Image Camera SN: 100%|██████████| 51/51 [00:00<00:00, 571.68it/s]

execute returned a non-zero exit status: 1
execute returned a non-zero exit status: 1
execute returned a non-zero exit status: 1
execute returned a non-zero exit status: 1
execute returned a non-zero exit status: 1
execute returned a non-zero exit status: 1
execute returned a non-zero exit status: 1
execute returned a non-zero exit status: 1
execute returned a non-zero exit status: 1
execute returned a non-zero exit status: 1
execute returned a non-zero exit status: 1
execute returned a non-zero exit status: 1
execute returned a non-zero exit status: 1
execute returned a non-zero exit status: 1
execute returned a non-zero exit status: 1
execute returned a non-zero exit status: 1
execute returned a non-zero exit status: 1
execute returned a non-zero exit status: 1
execute returned a non-zero exit status: 1
execute returned a non-zero exit status: 1
execute returned a non-zero exit status: 1
execute returned a non-zero exit status: 1
execute returned a non-zero exit status: 1
execute ret